In [1]:
from copy import deepcopy
import pandas as pd
from geopy.geocoders import Nominatim
import time
import numpy as np
import copy
from sklearn.cluster import KMeans
import pickle
import math

from Google_helper import getGoogleTimeDistance

In [3]:
"""
We used 20 addresses and 3 mail trucks as an example, but you can use any number of addresses and mail trucks that you want.

Just make sure you have enough credit on your Google account if you want to try a huge number of addresses. :)
"""

# This is the number of mail trucks that are available.
n_trucks = 3

# This is the list of all the addresses that we want to deliver mail to. You can read them from a CSV file or place them in a list here.
df = pd.read_csv("delivery_addresses.csv")

addrs = df['addresses'].values

"""
addrs = [
    "2701 Riverside Dr, Ottawa",
    "2515 Bank St, Ottawa",
    "30 Haxby Private, Ottawa",
    "116 Sai Crescent",
    "214 Plymouth St, Ottawa",
    "266 Lorry Greenberg Dr, Ottawa",
    "800 King Edward Ave, Ottawa",
    "2210 Bank St, Ottawa",
    "2161 Thurston Dr, Ottawa",
    "23 Ella Street, Ottawa",
    "11 Oblats Avenue, Ottawa",
    "422 Byron Avenue, Ottawa",
    "1680 Fisher Avenue, Ottawa",
    "825 Cahill Drive, Ottawa",
    "1125 Colonel By Dr, Ottawa",
    "54 Hawk Crescent, Ottawa",
    "1380 Prince of Wales Drive, Ottawa",
    "75 Laurier Ave. E, Ottawa, Ontario, Canada",
    "14 Waller St, Ottawa, Ontario, Canada",
    "363 Rideau St, Ottawa, Ontario, Canada",
]
"""

In [10]:
# You may supply your API key in a text file in the CWD.
with open("API_key.txt", "r") as f:
    GMAP_API_KEY = f.read()

"""
This gets the latitude and longitude coordinates of each address, which are needed for our clustering algorithm to compute distances.

inputs: str address
outputs: tuple[float]  result_loc
description: converts address to into latitude and longitude
"""
def coords_from_address(address):
    geolocator = Nominatim(user_agent = "test")
    location = geolocator.geocode(address)
    result_loc = (location.latitude, location.longitude)
    return result_loc

def write_gps_to_csv(gps_list, filename):
    df = pd.DataFrame(gps_list, columns=['Latitude', 'Longitude'])
    df.to_csv(filename, index=False)
    
#write_gps_to_csv(all_coords, "coords.csv")

In [11]:
# Addresses are converted into latitude and longitude coordinates so we can perform clustering on them.
coords = []

for addr in addrs:
    print(addr)
    coords.append(coords_from_address(addr))
    time.sleep(1)




2701 Riverside Dr, Ottawa
2515 Bank St, Ottawa
30 Haxby Private, Ottawa
116 Sai Crescent
214 Plymouth St, Ottawa
266 Lorry Greenberg Dr, Ottawa
800 King Edward Ave, Ottawa
2210 Bank St, Ottawa
2161 Thurston Dr, Ottawa
23 Ella Street, Ottawa
11 Oblats Avenue, Ottawa
422 Byron Avenue, Ottawa
1680 Fisher Avenue, Ottawa
825 Cahill Drive, Ottawa
1125 Colonel By Dr, Ottawa
54 Hawk Crescent, Ottawa
1380 Prince of Wales Drive, Ottawa
75 Laurier Ave. E, Ottawa, Ontario, Canada
14 Waller St, Ottawa, Ontario, Canada
363 Rideau St, Ottawa, Ontario, Canada


In [ ]:
"""
#This gets the time and distance matrix from the supplied addresses.

All pairwise combinations of addresses have their corresponding distances and travel times (factoring in weather and traffic) calculated byt the Google Distance Matrix API.

This information is used in our route optimization algorithm.
"""
T_mat, D_mat = getGoogleTimeDistance(addrs, GMAP_API_KEY)


In [36]:
#Pickle the saved matrices so we don't have to rerun the API and use more of our quota.
with open('T_mat','wb') as f1: 
    pickle.dump(T_mat, f1)

with open('T_mat','rb') as f2:
    new_T_mat = pickle.load(f2)
    
with open('D_mat','wb') as f3: 
    pickle.dump(D_mat, f3)

with open('D_mat','rb') as f4:
    new_D_mat = pickle.load(f4)

In [36]:
"""
This uses the K-means clustering algorithm to find the best way to allocate addresses given the number of trucks available.

Since the addresses have been converted to latitude/longitude coordinates, the K-means algorithm can compute distances between them and cluster them based on that.

We set k to the number of mail trucks, so we get a number of allocations equal to the number of mail trucks.

inputs: list(tuple) delivery coordinates, int n_trucks
outputs: dict[list[int]] Allocation
determines which trucks go where, referencing index of original address list and D_mat
"""
def assign_destinations(delivery_coordinates, addresses, n_trucks):
    coords_array = np.array(delivery_coordinates)
    kmeans = KMeans(n_clusters=n_trucks).fit(coords_array) 
    cluster_map = pd.DataFrame()
    
    #add one because the start location isnt counted
    cluster_map['destination_index'] = np.arange(len(coords_array))
    cluster_map['cluster'] = kmeans.labels_
    
    Allocation = {}
    for c_idx in set(cluster_map["cluster"]):
        location_details = []
        
        destination_idxs = list(cluster_map[cluster_map["cluster"]==c_idx]["destination_index"])
        for i in destination_idxs:
            
            loc_detail = (i+1, delivery_coordinates[i], addresses[i])
            
            location_details.append(loc_detail)
            
        Allocation[c_idx] = location_details
        #print(Allocation)
        
    return Allocation

"""
These helper methods are needed because our route planning algorithm doesn't work if the time to get from point A to point B isn't identical to the time to get from point B to point A,
which it often isn't in the real world due to one-way streets and other factors.

This is a flaw in our algorithm which we did not have time to fix. For now, we compute these times, choose the greatest of the two, and make both identical
to it. This results in a symmetric time matrix.
"""
def make_symmetrical(matrix):
    n = len(matrix)
    for i in range(n):
        for j in range(i+1, n):
            if matrix[i][j] > matrix[j][i]:
                matrix[j][i] = matrix[i][j]
            else:
                matrix[i][j] = matrix[j][i]
    return matrix

def sub_DT_mats(Allocation):
    """
    input dict[list[int]] Allocation
    output:  dict[nparray] T_mats, dict[nparray] D_mats
    description: calculates distance and time matrices for allocation with distance and time matrices
    """
    T_mats = {}
    D_mats = {}
    for t in Allocation.keys():

        #start with canada post address
        addresses = ["2701 Riverside Dr, Ottawa"]
        for loc_detail in Allocation[t]:
            addresses.append(loc_detail[2])
        T_mat, D_mat = getGoogleTimeDistance(addresses, GMAP_API_KEY)
        T_mats[t] = make_symmetrical(T_mat)
        D_mats[t] = make_symmetrical(D_mat)
        
    return T_mats, D_mats

def initial_configs(Allocation):
    result = copy.deepcopy(Allocation)
    
    
    for t in result.keys():
        for i in range(len(result[t])):
            result[t][i]=i+1
        result[t].insert(0,0)
    return result


In [37]:
#get delivery coordinates and addresses, and set the number of trucks to 3.
delivery_coords = coords[1:]
delivery_addrs = addrs[1:]

allocated_destinations = assign_destinations(delivery_coords, delivery_addrs, n_trucks)
start_configs = initial_configs(allocated_destinations)
T_mats, D_mats = sub_DT_mats(allocated_destinations)

In [39]:
"""
This is an implementation of the Metropolis travelling salesman algorithm.

Given a set of addresess and a matrix containing information about how long it takes to get from each address to each of the other addresses,
it finds the quickest route where each address will be visited once.

This finds the optimal delivery route for each mail truck.
"""
def calc_route_length(config, D_mat):
    
    route_length = 0
    #print(D_mat)
    #For each city, calculate the distance between it and previous.
    for i in range(len(config)):
        start_idx = config[i-1]
        end_idx = config[i]
        
        d = D_mat[start_idx][end_idx]
        route_length = route_length + d
        
    return route_length

#Define a function to generate a trial configuration where we swap the order of visiting two addresses.
def generate_trial_config(config):
    
    #Make a deep copy so we don't accidentally mess up our config:
    trial_config = deepcopy(config)
    
    #Generate one random address to switch:
    first_swap = np.random.randint(0, len(config))

    #Assign this value to our second swap:
    second_swap = first_swap
    
    #Generate a second random address to switch, making sure it's different from our first:
    while second_swap == first_swap:
        second_swap = np.random.randint(0, len(config))
    
    #Now swap our indices:
    hold_value = deepcopy(trial_config[first_swap])
    trial_config[first_swap] = trial_config[second_swap]
    trial_config[second_swap] = hold_value
    
    return trial_config, first_swap, second_swap

def calc_distance_change(config, trial_config, first_swap, second_swap, D_mat):
    
    #Calculate original distance:
    idx_i = config[first_swap]

    #Distance between first swap -1:
    idx_1_min = config[first_swap-1]
    
    d_one_min = D_mat[idx_i][idx_1_min]

    #Distance between first swap +1, set our bounds on our array of values:
    if (first_swap + 1) > (len(config)-1):
        swap_max = 0
    else:
        swap_max = first_swap +1
    idx_1_max = config[swap_max]
    
    d_one_max = D_mat[idx_1_max][idx_i]

    #Distance between second swap -1:
    idx_j = config[second_swap]
    idx_2_min = config[second_swap-1]
    
    d_two_min = D_mat[idx_j][idx_2_min]

    #Distance between second swap +1, set our bounds on array of values:
    if (second_swap + 1) > (len(config)-1):
        swap_max = 0
    else:
        swap_max = second_swap +1
        
    idx_2_max = config[swap_max]
    
    d_two_max = D_mat[idx_2_max][idx_j]
    
    #Now we have our original distance that navigating to these points took:
    original_distance = d_one_min + d_one_max + d_two_min + d_two_max
    
    #Now repeat for the trial configuration...

    #Calculate original distance:
    idx_i = trial_config[first_swap]
    
    #Distance between first swap -1:
    idx_1_min =  trial_config[first_swap-1]
   
    d_one_min = D_mat[idx_i][idx_1_min]

    #Distance between first swap +1, set our bounds on our array of values:
    if (first_swap + 1) > (len(trial_config)-1):
        swap_max = 0
    else:
        swap_max = first_swap +1
    idx_1_max = trial_config[swap_max]
    
    d_one_max = D_mat[idx_1_max][idx_i]
    
    #Distance between second swap -1:
    idx_j = trial_config[second_swap]
    idx_2_min = trial_config[second_swap-1]
   
    d_two_min = D_mat[idx_j][idx_2_min]

    #Distance between second swap +1, set our bounds on array of values:
    if (second_swap + 1) > (len(config)-1):
        swap_max = 0
    else:
        swap_max = second_swap +1
    idx_2_max = trial_config[swap_max]
    
    d_two_max = D_mat[idx_2_max][idx_j]

    #Now we have our new distance that navigating to these points took:
    new_distance = d_one_min + d_one_max + d_two_min + d_two_max
    
    #Calculate our change in distance:
    delta_d = new_distance - original_distance
    
    return delta_d

#Define a function to calculate our probability and evaluate it:
def evaluate_route_change(delta_d, T):
    
    #Calculate our probability:
    P = math.exp((-1*delta_d)/(T))

    #Geneate uniform random variable:
    u = np.random.uniform()

    #Estimate whether we're keeping this or not:
    if u <= P:
        accept = True
    else:
        accept = False
    
    return accept


def minimize_route(config, T, roc_tol, mean_samples, D_mat):
    
    #Create counter for our number of steps, and initial large value for rate of change:
    numsteps = 0
    roc = 10*roc_tol
    
    #Find our initial route length - we'll need this later...
    route_length = calc_route_length(config, D_mat)

    #Create array to store new route lengths:
    route_lengths = []
    
    # Want rate of change of our config over some number of steps to be small...
    while roc > roc_tol:
        
        #Generate trial configuration and calulate change in distance:
        trial_config, first_swap, second_swap = generate_trial_config(config)
        delta_d = calc_distance_change(config, trial_config, first_swap, second_swap, D_mat)
        
        #Evaluate if we are keeping our new configuration or not...
        if delta_d <= 0:
            config = deepcopy(trial_config)
            route_length = route_length + delta_d
            route_lengths.append(route_length)

            #Increment numsteps counter:
            numsteps = numsteps +1
        else:
            accept = evaluate_route_change(delta_d, T)
            if accept == True:
                config = deepcopy(trial_config)
                route_length = route_length + delta_d
                route_lengths.append(route_length)

                #Increment numsteps counter:
                numsteps = numsteps +1
            else: 
                pass
        
        #If we have more than 100 steps, evaluate our mean route length over the last 50 steps,
        #compare to the 50 before, and see what our rate of change is.
        if numsteps > (mean_samples*2):
            new_mean = np.mean(route_lengths[numsteps-mean_samples:])
            old_mean = np.mean(route_lengths[numsteps-(mean_samples*2):numsteps-mean_samples])
            roc = abs(new_mean - old_mean)
            
    return config, route_lengths



def travelling_salesman(initial_config, initial_T, delta_T, roc_tol_at_T, roc_tol, mean_samples, failsafe, D_mat):

    while True:

        #Do some math with our original route, including calculating route lengths etc.
        config = initial_config
        old_route_length = calc_route_length(config, D_mat)
        
        old_route_lengths = []
        old_route_lengths.append(old_route_length)

        #Create set of unique routes.
        unique_routes = set(np.around(old_route_lengths,3))

        #Set an initial roc we can evaluate against.
        roc = 10*roc_tol

        #Set our initial T.
        T = initial_T

        print(T)

        #Run while our roc is greater than our tol, and our temperature is above 0:
        while (roc > roc_tol) and (T > 0):

            #Minimize route at current T:
            config, route_lengths =  minimize_route(config, T, roc_tol, mean_samples, D_mat)

            #If we've come up with a shorter route:
            if len(route_lengths) != 0:

                #Calculate change in mean:
                old_mean = np.mean(old_route_lengths)
                new_mean = np.mean(route_lengths)
                roc = abs(new_mean - old_mean)

                #If our roc is still greater:
                if roc > roc_tol:

                    #Swap over and update our recent routes...
                    old_route_lengths = deepcopy(route_lengths)
                    temp_set_routes = set(np.around(old_route_lengths,3))
                    unique_routes = unique_routes.union(temp_set_routes)
                else:
                    pass
            else:
                pass
            
            #Increment T downwards...
            T = T - delta_T
            print(T)
            
            #If T goes below zero...
            if T <= 0:
                route_lengths.append(old_route_lengths[-1])
            else:
                pass
        
        #Sort our set of routes so we can use these in failsafe.
        sorted_routes = sorted(unique_routes)
            
        #If we're not using the failsafe, or if our route is as short as the shortest possible route after hitting roc tolerance, break the loop.
        if (failsafe == False) or (round(route_lengths[-1],3) <= (round(sorted_routes[0],3)+0.001)):
            break
    
    return config, unique_routes, route_lengths[-1]


In [6]:
"""
These are some quick indexing fixes.
"""
config0 = [(168, 15), (493, 72), (22, 299), (466, 75), (360, 263)]
config_idx = [0,1,2,3,4]
D_mat = np.zeros((len(config0), len(config0)))
for i in range(len(config0)):
    for k in range(len(config0)):
        if i !=k:
            D_mat[i][k] = np.sqrt((config0[i][0]-config0[k][0])**2+(config0[i][1]-config0[k][1])**2)

In [ ]:
# Run the route planning algorithm for all the trucks.
initial_T = 25
delta_T = 0.5
roc_tol_at_T = 0.1
roc_tol = 0.1
mean_samples = 100
failsafe = False

T_seq ={}
for i in range(0, len(start_configs)):
    initial_config = start_configs[i]
    D_mat = T_mats[i]

    config, unique_routes, final_route_length = travelling_salesman(initial_config, initial_T, delta_T, roc_tol_at_T, roc_tol, mean_samples, failsafe, D_mat)
    print(config)
    print(final_route_length)

    T_seq[i] = config

In [40]:
"""
# Run the route planning algorithm for the first truck.
initial_T = 25
delta_T = 0.5
roc_tol_at_T = 0.1
roc_tol = 0.1
mean_samples = 100
failsafe = False
initial_config = start_configs[0]
D_mat = T_mats[0]

config, unique_routes, final_route_length = travelling_salesman(initial_config, initial_T, delta_T, roc_tol_at_T, roc_tol, mean_samples, failsafe, D_mat)
print(config)
print(final_route_length)
"""

25
24.5
24.0
23.5
23.0
22.5
22.0
21.5
21.0
20.5
20.0
19.5
19.0
18.5
18.0
17.5
17.0
16.5
16.0
15.5
15.0
14.5
[0, 5, 2, 4, 6, 3, 1, 8, 7]
3396


In [41]:
"""
# Run the route planning algorithm for the second truck.
initial_T = 25
delta_T = 0.5
roc_tol_at_T = 0.1
roc_tol = 0.1
mean_samples = 100
failsafe = False
initial_config = start_configs[1]
D_mat = T_mats[1]

config, unique_routes, final_route_length = travelling_salesman(initial_config, initial_T, delta_T, roc_tol_at_T, roc_tol, mean_samples, failsafe, D_mat)
print(config)
print(final_route_length)
"""

25
24.5
24.0
23.5
23.0
22.5
22.0
21.5
21.0
20.5
20.0
19.5
19.0
18.5
18.0
17.5
17.0
16.5
16.0
15.5
15.0
14.5
14.0
13.5
13.0
12.5
12.0
11.5
11.0
10.5
10.0
[0, 1, 3, 6, 7, 5, 2, 4]
3383


In [42]:
"""
# Run the route planning algorithm for the third truck. This was stopped prematurely once we had all the needed data, as we got a little impatient and wanted to go home for the night. :)
initial_T = 25
delta_T = 0.5
roc_tol_at_T = 0.1
roc_tol = 0.1
mean_samples = 100
failsafe = False
initial_config = start_configs[2]
D_mat = T_mats[2]

config, unique_routes, final_route_length = travelling_salesman(initial_config, initial_T, delta_T, roc_tol_at_T, roc_tol, mean_samples, failsafe, D_mat)
print(config)
print(final_route_length)
"""

25
24.5
24.0
23.5
23.0
22.5
22.0
21.5
21.0
20.5
20.0
19.5
19.0
18.5
18.0
17.5
17.0
16.5
16.0
15.5
15.0
14.5
14.0
13.5
13.0
12.5
12.0
11.5
11.0
10.5
10.0
9.5
9.0
8.5
8.0
7.5
7.0
6.5
6.0


KeyboardInterrupt: 

In [45]:
"""
# The ideal routes were printed to the console, and we've hardcoded them here for further processing.
T0_seq = [0, 5, 2, 4, 6, 3, 1, 8, 7]
T1_seq = [0, 1, 3, 6, 7, 5, 2, 4]
T2_seq = [3, 4, 2, 0, 1]

T_seq ={0:T0_seq, 1: T1_seq, 2: T2_seq}
"""

In [164]:
# Since each mail truck receives a sequence based on its local cluster, we use these sequences to find the addresses in the address list and get the waypoints.

coord_seq = {}
all_loc_detail = [(0, (45.376031499999996, -75.69003843087424), "2701 Riverside Dr, Ottawa")]


for t in T_seq.keys():
    coord_seq[t] = []
    all_loc_detail = all_loc_detail + allocated_destinations[t]
    
all_loc_detail.sort(key=lambda x: x[0])
for t in T_seq.keys():
    print("truck: "+str(t))
    
    for i in T_seq[t]:

        if i != 0:
            #subtract by 1
            k = i-1
            global_index = allocated_destinations[t][k][0]
            
        if i == 0:
            global_index = 0
            
            
        coord_seq[t].append(all_loc_detail[global_index][1]) 
        print(global_index)
    print(t)
    

truck: 0
0
7
2
5
8
3
1
15
13
0
truck: 1
0
4
9
18
19
17
6
10
1
truck: 2
14
16
12
0
11
2


In [166]:
# Write the waypoints for each truck to CSV. These will be used to plot our routes on a map.
for t in coord_seq.keys():
    f_name = "t_"+str(t)+".csv"
    df = pd.DataFrame(coord_seq[t], columns = ["latitude", "longitude"])
    df.to_csv(f_name, index=False)
 